# TensorFlow Linear and DNN Classifiers

## predict whether a patient has diabetes or not

In [1]:
#imports
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

#data
diabetes = pd.read_csv('../Tensorflow-Bootcamp-master/02-TensorFlow-Basics/pima-indians-diabetes.csv')
print(diabetes.columns, '\n')
cols_to_norm = ['Number_pregnant', 'Glucose_concentration', 'Blood_pressure', 'Triceps',
       'Insulin', 'BMI', 'Pedigree']
diabetes[cols_to_norm] = diabetes[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
num_preg = tf.feature_column.numeric_column('Number_pregnant')
plasma_gluc = tf.feature_column.numeric_column('Glucose_concentration')
dias_press = tf.feature_column.numeric_column('Blood_pressure')
tricep = tf.feature_column.numeric_column('Triceps')
insulin = tf.feature_column.numeric_column('Insulin')
bmi = tf.feature_column.numeric_column('BMI')
diabetes_pedigree = tf.feature_column.numeric_column('Pedigree')
age = tf.feature_column.numeric_column('Age')
assigned_group = tf.feature_column.categorical_column_with_vocabulary_list('Group',['A','B','C','D'])
age_buckets = tf.feature_column.bucketized_column(age, boundaries=[20,30,40,50,60,70,80])

#split
x_data = diabetes.drop('Class',axis=1)
labels = diabetes['Class']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.33, random_state=101)

#LinearClassifier model
feat_cols = [num_preg ,plasma_gluc,dias_press ,tricep ,insulin,bmi,diabetes_pedigree ,assigned_group, age_buckets]
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)
model = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)
model.train(input_fn=input_func,steps=1000)

#evaluation
eval_input_func_lr = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)
results_LR = model.evaluate(eval_input_func_lr)

#predict
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)
predictions = model.predict(pred_input_func)
list(predictions)

#data
embedded_group_column = tf.feature_column.embedding_column(assigned_group, dimension=4)

#DNN model
feat_cols = [num_preg ,plasma_gluc,dias_press ,tricep ,insulin,bmi,diabetes_pedigree ,embedded_group_column, age_buckets]
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols,n_classes=2)
dnn_model.train(input_fn=input_func,steps=1000)

#evaluation
eval_input_func_dnn = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)
results_DNN = dnn_model.evaluate(eval_input_func_dnn)
print('\n evaluation results: \n')
results_DNN

Index(['Number_pregnant', 'Glucose_concentration', 'Blood_pressure', 'Triceps',
       'Insulin', 'BMI', 'Pedigree', 'Age', 'Class', 'Group'],
      dtype='object') 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_tf_random_seed': 1, '_log_step_count_steps': 100, '_session_config': None, '_model_dir': 'C:\\Users\\PERGO_~1\\AppData\\Local\\Temp\\tmp7gzni9xo', '_keep_checkpoint_max': 5, '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\PERGO_~1\AppData\Local\Temp\tmp7gzni9xo\model.ckpt.
INFO:tensorflow:step = 1, loss = 6.93147
INFO:tensorflow:global_step/sec: 275.292
INFO:tensorflow:step = 101, loss = 6.96317 (0.363 sec)
INFO:tensorflow:global_step/sec: 343.069
INFO:tensorflow:step = 201, loss = 5.60476 (0.291 sec)
INFO:tensorflow:global_step/sec: 283.588
INFO:tensorflow:step = 301, lo

{'accuracy': 0.72834647,
 'accuracy_baseline': 0.65748036,
 'auc': 0.81516272,
 'auc_precision_recall': 0.66099477,
 'average_loss': 0.52433401,
 'global_step': 1000,
 'label/mean': 0.34251967,
 'loss': 5.1223397,
 'prediction/mean': 0.42819217}